# <font color='Purple'>Ciência dos Dados - Na Prática</font>

# <font color='blue'>Criando um Classificador de Documentos com PLN e Naive Bayes</font>

http://scikit-learn.org/stable/modules/naive_bayes.html

# Multinomial Naive Bayes - Scikit-Learn

O classificador Multinomial Naive Bayes é adequado para classificação com variáveis discretas (por exemplo, contagens de palavras para a classificação de texto). A distribuição multinomial normalmente requer contagens de entidades inteiras. No entanto, na prática, contagens fracionadas (contagem de palavras) como tf-idf também podem funcionar.

http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

# Classificador de Documentos

> Ex: Escritórios de Advocacia (milhões de petições...), Diagnósticos de Exames Laboratoriais (Sabin , Exame) - Pesquisas Acadêmicas, Relatório Financeiro de empresas de Capital aberto

      



http://qwone.com/~jason/20Newsgroups/

In [0]:
# Imports
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [0]:
fetch_20newsgroups

In [0]:
# Definindo as categorias 
# (usando apenas 4 de um total de 20 disponível para que o processo de classificação seja mais rápido)
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

In [0]:
# Treinamento
twenty_train = fetch_20newsgroups(subset = 'train', categories = categories, shuffle = True, random_state = 42)

In [0]:
# Classes
twenty_train.target_names

In [0]:
len(twenty_train.data)

In [0]:
# Visualizando alguns dados (atributos)
print("\n".join(twenty_train.data[0].split("\n")[:10]))

In [0]:
# Visualizando variáveis target
print(twenty_train.target_names[twenty_train.target[0]])

In [0]:
# O Scikit-Learn registra os labels como array de números, a fim de aumentar a velocidade
twenty_train.target[:10]

In [0]:
# Visualizando as classes dos 10 primeiros registros
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

# Bag of Words (Saco de Palavras)

In [15]:
# Tokenizing
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
count_vect.vocabulary_.get(u'algorithm')
X_train_counts.shape

(2257, 35788)

In [14]:
# De ocorrências a frequências - Term Frequency times Inverse Document Frequency (Tfidf)
tf_transformer = TfidfTransformer(use_idf = False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

In [16]:
# Mesmo resultado da célula anterior, mas combinando as funções
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

In [0]:
# Criando o modelo Multinomial
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [24]:
# Previsões
docs_new = ['Darwin fish bumper stickers']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'Darwin fish bumper stickers' => alt.atheism


In [0]:
# Criando um Pipeline - Classificador Composto
# vectorizer => transformer => classifier 
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [0]:
# Fit
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

In [27]:
# Acurácia do Modelo
twenty_test = fetch_20newsgroups(subset = 'test', categories = categories, shuffle = True, random_state = 42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)    

0.8348868175765646

In [28]:
# Métricas
print(metrics.classification_report(twenty_test.target, predicted, target_names = twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.97      0.60      0.74       319
         comp.graphics       0.96      0.89      0.92       389
               sci.med       0.97      0.81      0.88       396
soc.religion.christian       0.65      0.99      0.78       398

              accuracy                           0.83      1502
             macro avg       0.89      0.82      0.83      1502
          weighted avg       0.88      0.83      0.84      1502



In [29]:
# Confusion Matrix
metrics.confusion_matrix(twenty_test.target, predicted)

array([[192,   2,   6, 119],
       [  2, 347,   4,  36],
       [  2,  11, 322,  61],
       [  2,   2,   1, 393]])

In [0]:
# Parâmetros para o GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
}

In [0]:
# GridSearchCV
gs_clf = GridSearchCV(text_clf, parameters, n_jobs = -1)

In [0]:
# Fit
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

In [42]:
# Teste
twenty_train.target_names[gs_clf.predict(['atheist paraphernalia'])[0]]

'alt.atheism'

In [43]:
# Score
gs_clf.best_score_        

0.9349999999999999

In [44]:
# Parâmetros utilizados
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.01
tfidf__use_idf: True
vect__ngram_range: (1, 2)


# Fim

![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTK4gQ9nhwHHaSXMHpeggWg7twwMCgb877smkRmtkmDeDoGF9Z6&usqp=CAU)

### #Tamojunto - Ciência dos Dados - <a href="http://facebook.com/cienciadosdadosbr">facebook.com/cienciadosdadosbr</a>

### #Scripts e Datasets - Comunidade Telegram <a href="https://t.me/cienciadosdados">https://t.me/cienciadosdados</a>

### #Mais Aulas como essa no YouTube <a href="https://www.youtube.com/watch?v=IaIc5oHd3II&t=1569s">https://www.youtube.com/watch?v=IaIc5oHd3II&t=1569s</a>

In [51]:
from IPython.core.display import HTML
HTML('<iframe width="300" height="200" src="https://www.youtube.com/embed/IaIc5oHd3II" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></iframe>')